# 0. Setup

Import packages and specify any important functions here.

In [15]:
# import standard python packages
import pandas as pd
import subprocess
import os
import dill

# add the utils and env directories to the path
import sys
sys.path.append('../../utils/')
sys.path.append('../../env/')

# import functions from utils directory files
from string_functions import *
from biofile_handling import *

# import paths to software installs from env
from install_locs import *

# 1. Download and describe data

This notebook collects data from the [Liao et al. 2022](https://www.sciencedirect.com/science/article/pii/S0092867418301168#sec4) Xenopus laevis adult cell atlas.

## Dataset description

- Data can be found at the [GEO Accession GSE195790](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE195790).
- This notebook previously collects data from one sample, ["Xenopus_brain_COL65"](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM6214268), for the purposes of data analysis and exploration.

In [16]:
################
# general info #
################

# Specify the name of the species folder in Amazon S3
species = 'Xenopus_laevis'

# Specify any particular identifying conditions, eg tissue type:
conditions = 'adultbrain'

# Specify url and other variables
genome_fasta_url = 'https://ftp.xenbase.org/pub/Genomics/JGI/Xenla9.2/XENLA_9.2_genome.fa.gz'
genome_version = 'XENLA_9.2'

annot_url = 'https://ftp.xenbase.org/pub/Genomics/JGI/Xenla9.2/XENLA_9.2_GCF.gff3'
gxc_url = 'https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM6214nnn/GSM6214268/suppl/GSM6214268_Xenopus_brain_COL65_dge.txt.gz'

###########
# runtime #
###########

protocol = 'curl'

species_prefix = prefixify(species)

# Specify folder as destination for file downloads
output_folder = '../../output/' + prefixify(species) + '_' + conditions + '/'

if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    
species_SampleDict = SampleDict(species, conditions, output_folder)

genome_fasta = GenomeFastaFile(
    filename = '',
    sampledict = species_SampleDict,
    version = genome_version,
    url = genome_fasta_url,
    protocol = protocol
)

annot = GenomeGffFile(
    filename = '',
    sampledict = species_SampleDict,
    GenomeFastaFile = genome_fasta,
    url = annot_url,
    protocol = protocol
)

gxc = GxcFile(
    filename = '',
    sampledict = species_SampleDict,
    GenomeFastaFile = genome_fasta,
    GenomeAnnotFile = annot,
    url = gxc_url,
    protocol = protocol
)


sample_Docket = Docket(species_SampleDict)
keyfiles = {
    'annot': annot,
    'genome_fasta': genome_fasta,
    'gxc': gxc
}
sample_Docket.add_keyfiles(keyfiles)

display(vars(sample_Docket))

{'species': 'Xenopus_laevis',
 'conditions': 'adultbrain',
 'directory': '../../output/Xlae_adultbrain/',
 'files': {},
 'annot': <biofile_handling.GenomeGffFile at 0x7fbcc9930130>,
 'genome_fasta': <biofile_handling.GenomeFastaFile at 0x7fb9d5c36740>,
 'gxc': <biofile_handling.GxcFile at 0x7fbcc9933c70>}

# 2. Load in the gxc matrix and get gene names

In [17]:
genes_matrix = pd.read_csv(sample_Docket.gxc.path, sep = '\t')
display(genes_matrix)

gxc_genes_list = pd.DataFrame({'gene_name':genes_matrix['GENE']})
display(gxc_genes_list)

,GENE,AACCTATTCATATAAGGG,CTCGCATCAAAGTTAACT,AACCTAGTATACTTCCGC,AACCTAAAAGTTCTGAAA,CTCGCACGCACCCTCCAT,ACGTTGTATTGTAGCGAG,ACGAGCATGCTTTAGTCG,AACCTAGTCCCGCCATCT,AACCTAGCGAATTAGAGA,...,TCACTTGTTGCCATGCTT,TCGGGTTGTCACACTTAT,TCGTAATCGTAAGTTGCC,TGTCACGAATTACACAAG,TGTGCGTACTTCTAGTCG,TTAACTATACAGTGGATG,TTGGACACTTATGATCTT,AAAGTTACTTATGCCCTC,AACCTACGCACCTGCGGA,AACCTATAGTCGCTGTGT
0,3.S,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,42Sp43.L,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,42Sp50.L,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AK6.L,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AK6.S,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25907,zyg11b.L,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25908,zyg11b.S,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25909,zzef1.S,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25910,zzz3.L,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0


,gene_name
0,3.S
1,42Sp43.L
2,42Sp50.L
3,AK6.L
4,AK6.S
...,...
25907,zyg11b.L
25908,zyg11b.S
25909,zzef1.S
25910,zzz3.L


# 3. Get mapping identifiers

In [18]:
# load in the original GFF-based annotation
models = pd.read_csv(sample_Docket.annot.path, skiprows = 6, header = None, sep = '\t', on_bad_lines = 'skip')
display(models)

attributes_column = 8

# Check the structure of fields in the GFF additional fields section
display(models[attributes_column][3])

/tmp/ipykernel_9309/510702444.py:2: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  models = pd.read_csv(sample_Docket.annot.path, skiprows = 6, header = None, sep = '\t', on_bad_lines = 'skip')


,0,1,2,3,4,5,6,7,8
0,chr1L,Gnomon,gene,105976,106484.0,.,-,.,ID=gene0;Dbxref=GeneID:108719298;Name=LOC10871...
1,chr1L,Gnomon,V_gene_segment,105976,106484.0,.,-,.,ID=id1;Parent=gene0;Dbxref=GeneID:108719298;gb...
2,chr1L,Gnomon,exon,106430,106484.0,.,-,.,ID=id2;Parent=id1;Dbxref=GeneID:108719298;gbke...
3,chr1L,Gnomon,exon,105976,106304.0,.,-,.,ID=id3;Parent=id1;Dbxref=GeneID:108719298;gbke...
4,chr1L,Gnomon,CDS,106430,106484.0,.,-,0,ID=cds0;Parent=gene0;Dbxref=GeneID:108719298;e...
...,...,...,...,...,...,...,...,...,...
1642208,##sequence-region Scaffold115627 1 200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1642209,##sequence-region Scaffold115628 1 200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1642210,##sequence-region Scaffold115629 1 200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1642211,##sequence-region Scaffold115630 1 200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


'ID=id3;Parent=id1;Dbxref=GeneID:108719298;gbkey=V_segment;gene=LOC108719298;model_evidence=Supporting evidence includes similarity to: 9 Proteins'

In [19]:
# Remove any rows with NaNs
models.dropna(inplace = True)

# Extract field and database cross-ref (dbxref) information into columns
models['field_dictionary'] = models[attributes_column].apply(convert_fields_to_dict_gff)
models['gene_name'] = [d.get('gene') for d in models['field_dictionary']]
models['Dbxref'] = [d.get('Dbxref') for d in models['field_dictionary']]
models['dbxref_dict'] = models['Dbxref'].apply(convert_dbxref_to_dict)

display(models)

,0,1,2,3,4,5,6,7,8,field_dictionary,gene_name,Dbxref,dbxref_dict
0,chr1L,Gnomon,gene,105976,106484.0,.,-,.,ID=gene0;Dbxref=GeneID:108719298;Name=LOC10871...,"{'ID': 'gene0', 'Dbxref': 'GeneID:108719298', ...",LOC108719298,GeneID:108719298,{'GeneID': '108719298'}
1,chr1L,Gnomon,V_gene_segment,105976,106484.0,.,-,.,ID=id1;Parent=gene0;Dbxref=GeneID:108719298;gb...,"{'ID': 'id1', 'Parent': 'gene0', 'Dbxref': 'Ge...",LOC108719298,GeneID:108719298,{'GeneID': '108719298'}
2,chr1L,Gnomon,exon,106430,106484.0,.,-,.,ID=id2;Parent=id1;Dbxref=GeneID:108719298;gbke...,"{'ID': 'id2', 'Parent': 'id1', 'Dbxref': 'Gene...",LOC108719298,GeneID:108719298,{'GeneID': '108719298'}
3,chr1L,Gnomon,exon,105976,106304.0,.,-,.,ID=id3;Parent=id1;Dbxref=GeneID:108719298;gbke...,"{'ID': 'id3', 'Parent': 'id1', 'Dbxref': 'Gene...",LOC108719298,GeneID:108719298,{'GeneID': '108719298'}
4,chr1L,Gnomon,CDS,106430,106484.0,.,-,0,ID=cds0;Parent=gene0;Dbxref=GeneID:108719298;e...,"{'ID': 'cds0', 'Parent': 'gene0', 'Dbxref': 'G...",LOC108719298,GeneID:108719298,{'GeneID': '108719298'}
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637051,##sequence-region Scaffold110469 1 208,Scaffold110469,Gnomon,gene,1.0,208.0,.,+,.,{},None,None,None
1637629,##sequence-region Scaffold111047 1 207,Scaffold111047,tRNAscan-SE,gene,88.0,170.0,.,+,.,{},None,None,None
1639032,##sequence-region Scaffold112450 1 204,Scaffold112450,BestRefSeq,gene,1.0,145.0,.,+,.,{},None,None,None
1640774,##sequence-region Scaffold114193 1 202,Scaffold114193,tRNAscan-SE,gene,63.0,135.0,.,+,.,{},None,None,None


# 4. Extract gene IDs for mapping to UniprotKB
Specify which set of identifiers will be use to query the [Uniprot ID Mapping Tool](https://www.uniprot.org/id-mapping) via API.

If using an identifier from the `dbxref_dict`, specify the name via string in the `dbxref_datafield` variable.

In [20]:
dbxref_datafield = 'GeneID'
datafield = ''

if dbxref_datafield != '':
    models.dropna(axis = 0, subset = ['dbxref_dict'], inplace = True)
    models[dbxref_datafield] = [d.get(dbxref_datafield) for d in models['dbxref_dict']]

    models_subset = models[['gene_name', dbxref_datafield]].dropna().drop_duplicates()
    display(models_subset)

elif datafield == 'gene_name':
    models_subset = models[['gene_name']].dropna().drop_duplicates()
    display(models_subset)

elif datafield != '':
    models_subset = models[['gene_name', datafield]].dropna().drop_duplicates()
    display(models_subset)

else:
    raise Exception('You must provide a data field for ID mapping.')

,gene_name,GeneID
0,LOC108719298,108719298
6,LOC108719307,108719307
12,LOC108697125,108697125
20,LOC108696185,108696185
26,LOC108696193,108696193
...,...,...
1571901,LOC108706234,108706234
1572166,ndufaf4.L,734768
1572435,LOC108706236,108706236
1572470,LOC108706237,108706237


# 5. Generate gene list file to query Uniprot ID Mapping API
Generate a text file ending in `_ids.txt` for sending to the ID mapping API.

In [21]:
datafield = dbxref_datafield if dbxref_datafield != '' else datafield

gene_list = models_subset[datafield].unique()

genelist_object = GeneListFile(
    filename = '', # filename is automatically generated from sampledict
    sampledict = species_SampleDict,
    sources = [sample_Docket.annot],
    genes = gene_list,
    identifier = datafield
    )

Wrote 36828 gene ids to ../../output/Xlae_adultbrain/Xlae_adultbrain_GeneID_ids.txt


# 6. Query Uniprot ID Mapping API
Specify the `from_type` variable based on the Uniprot name of the identifier.
The table below lists some databases and the `from_type` string that the API accepts for that datatype.

For some reason, the ID mapping API usually returns "resource not found" the first time you use it.
Try re-running this cell if you receive that error.


| datatype | `from_type` string | description |
| ---: | :--- | :--- |
| Mouse Genome Informatics | `MGI` | ID starts with `MGI:` |
| Zebrafish Information Network | `ZFIN` | ID starts with `ZDB-GENE-` |
| NCBI Gene ID | `GeneID` | varies, usually numeric string |

In [22]:
from_type = 'GeneID'
to_type = 'UniProtKB'

uniprot_idmm_object = genelist_object.get_uniprot_ids(ID_MAPPER_LOC, from_type, to_type)
sample_Docket.add_keyfiles({'uniprot_idmm': uniprot_idmm_object})

uniprot_idmm = pd.read_csv(sample_Docket.uniprot_idmm.path, sep = '\t')
display(uniprot_idmm)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  329k    0    52  100  329k     25   164k  0:00:02  0:00:02 --:--:--  164k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    24    0    24    0     0     29      0 --:--:-- --:--:-- --:--:--    29


,From,Entry,Entry Name,Reviewed,Protein names,Gene Names,Organism,Length
0,494796,Q63ZJ1,Q63ZJ1_XENLA,unreviewed,"LOC494796 protein (succinate-CoA ligase, alpha...",suclg1.L galpha LOC494796 mtdps9 sucla1 suclg1...,Xenopus laevis (African clawed frog),195
1,444710,Q6GLP0,CTNA2_XENLA,reviewed,Catenin alpha-2 (Alpha N-catenin),ctnna2,Xenopus laevis (African clawed frog),966
2,734444,Q52L02,Q52L02_XENLA,unreviewed,Receptor-type tyrosine-protein phosphatase alp...,ptpre.L ptpra ptpre,Xenopus laevis (African clawed frog),819
3,380428,Q8AVT3,Q8AVT3_XENLA,unreviewed,Paip2-prov protein (poly(A) binding protein in...,paip2b.L paip2 paip2b,Xenopus laevis (African clawed frog),125
4,779272,Q05AW8,Q05AW8_XENLA,unreviewed,Vacuolar proton pump subunit B (V-ATPase subun...,atp6v1b1.L atp6v1b1 MGC154483,Xenopus laevis (African clawed frog),506
...,...,...,...,...,...,...,...,...
11224,100335048,Q2ABY0,Q2ABY0_XENLA,unreviewed,ADP-ribosylation factor-like protein 6,arl6.S ARL6 arl6,Xenopus laevis (African clawed frog),186
11225,495480,Q5U4K1,Q5U4K1_XENLA,unreviewed,LOC495480 protein (uncharacterized protein LOC...,LOC495480,Xenopus laevis (African clawed frog),215
11226,398255,Q90YY8,Q90YY8_XENLA,unreviewed,Marginal coil Xmc (marginal coil Xmc L homeolog),Xmc xmc xmc.L xmc.S,Xenopus laevis (African clawed frog),515
11227,447434,Q66JA7,Q66JA7_XENLA,unreviewed,Pyruvate dehydrogenase E1 component subunit al...,pdha1.S pdha pdha1 pdha1-a pdha1-b pdhce1a phe1a,Xenopus laevis (African clawed frog),400


# 7. Extract results and generate Uniprot IDMM
Generates an idmm that links `gene_name`, the `dbxref_datafield` seleted above, and `uniprot_id` returned by API.

In [23]:
uniprot_idpairs = uniprot_idmm[['From', 'Entry']]
uniprot_idpairs.rename(columns = {'From': datafield, 'Entry': 'uniprot_id'}, inplace = True)
uniprot_idpairs[datafield] = uniprot_idpairs[datafield].astype(str)
display(uniprot_idpairs)

uniprot_output_idmm = models_subset.merge(uniprot_idpairs, on = datafield)
display(uniprot_output_idmm)

/tmp/ipykernel_9309/899271637.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniprot_idpairs.rename(columns = {'From': datafield, 'Entry': 'uniprot_id'}, inplace = True)
/tmp/ipykernel_9309/899271637.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniprot_idpairs[datafield] = uniprot_idpairs[datafield].astype(str)


,GeneID,uniprot_id
0,494796,Q63ZJ1
1,444710,Q6GLP0
2,734444,Q52L02
3,380428,Q8AVT3
4,779272,Q05AW8
...,...,...
11224,100335048,Q2ABY0
11225,495480,Q5U4K1
11226,398255,Q90YY8
11227,447434,Q66JA7


,gene_name,GeneID,uniprot_id
0,suclg1.S,494796,Q63ZJ1
1,ctnna2.S,444710,Q6GLP0
2,ptpre.L,734444,Q52L02
3,paip2,380428,Q8AVT3
4,atp6v1b1.L,779272,Q05AW8
...,...,...,...
11224,arl6,100335048,Q2ABY0
11225,LOC495480,495480,Q5U4K1
11226,xmc.S,398255,Q90YY8
11227,pdha1.S,447434,Q66JA7


In [24]:
# generate a filename and file for the idmm
uniprot_output_idmm_filename = '_'.join([species_prefix, conditions, 'uniprot-idmm.tsv'])
uniprot_output_idmm_object = IdmmFile(uniprot_output_idmm_filename, species_SampleDict, kind = 'uniprot_idmm', sources = [sample_Docket.annot])

# save to file and add to the Docket
uniprot_output_idmm.to_csv(uniprot_output_idmm_object.path, sep = '\t')
sample_Docket.add_keyfile(uniprot_output_idmm_object, 'uniprot_idmm')

# 8. Convert GFF to GTF

In [25]:
# convert the GFF file to GTF using gffread
models_asgtf = sample_Docket.annot.to_gtf(GFFREAD_LOC)

In [26]:
# load the newly-generated GTF file as a dataframe
models_asgtf_df = pd.read_csv(models_asgtf.path, skiprows = 0, header = None, sep = '\t')

display(models_asgtf_df)
display(models_asgtf_df[attributes_column][1])

,0,1,2,3,4,5,6,7,8
0,chr1L,Gnomon,transcript,105976,106484,.,-,.,"transcript_id ""id1""; gene_id ""gene0""; gene_nam..."
1,chr1L,Gnomon,exon,105976,106304,.,-,.,"transcript_id ""id1""; gene_id ""gene0""; gene_nam..."
2,chr1L,Gnomon,exon,106430,106484,.,-,.,"transcript_id ""id1""; gene_id ""gene0""; gene_nam..."
3,chr1L,Gnomon,CDS,105976,106304,.,-,2,"transcript_id ""id1""; gene_id ""gene0""; gene_nam..."
4,chr1L,Gnomon,CDS,106430,106484,.,-,0,"transcript_id ""id1""; gene_id ""gene0""; gene_nam..."
...,...,...,...,...,...,...,...,...,...
1499203,MT,RefSeq,CDS,16249,17388,.,+,0,"transcript_id ""gene37282""; gene_name ""CYTB"";"
1499204,MT,RefSeq,transcript,17388,17457,.,+,.,"transcript_id ""rna62490""; gene_id ""rna62490"""
1499205,MT,RefSeq,exon,17388,17457,.,+,.,"transcript_id ""rna62490"";"
1499206,MT,RefSeq,transcript,17485,17553,.,-,.,"transcript_id ""rna62491""; gene_id ""rna62491"""


'transcript_id "id1"; gene_id "gene0"; gene_name "LOC108719298";'

In [27]:
# Use a custom function to extract useful fields from the additional fields section (column 8)
# Pull from that dict to fill in additional useful columns
models_asgtf_df['field_dictionary'] = models_asgtf_df[attributes_column].apply(convert_fields_to_dict_gtf)
models_asgtf_df['gene_name'] = [d.get('gene_name') for d in models_asgtf_df['field_dictionary']]
models_asgtf_df['gene_id'] = [d.get('gene_id') for d in models_asgtf_df['field_dictionary']]
models_asgtf_df['transcript_id'] = [d.get('transcript_id') for d in models_asgtf_df['field_dictionary']]

# Remove CDS annotations because they interfere with TransDecoder cDNA generation
models_asgtf_df = models_asgtf_df[models_asgtf_df[2] != 'CDS']
display(models_asgtf_df)

,0,1,2,3,4,5,6,7,8,field_dictionary,gene_name,gene_id,transcript_id
0,chr1L,Gnomon,transcript,105976,106484,.,-,.,"transcript_id ""id1""; gene_id ""gene0""; gene_nam...","{'transcript_id': 'id1', 'gene_id': 'gene0', '...",LOC108719298,gene0,id1
1,chr1L,Gnomon,exon,105976,106304,.,-,.,"transcript_id ""id1""; gene_id ""gene0""; gene_nam...","{'transcript_id': 'id1', 'gene_id': 'gene0', '...",LOC108719298,gene0,id1
2,chr1L,Gnomon,exon,106430,106484,.,-,.,"transcript_id ""id1""; gene_id ""gene0""; gene_nam...","{'transcript_id': 'id1', 'gene_id': 'gene0', '...",LOC108719298,gene0,id1
5,chr1L,Gnomon,transcript,136318,136954,.,-,.,"transcript_id ""id4""; gene_id ""gene1""; gene_nam...","{'transcript_id': 'id4', 'gene_id': 'gene1', '...",LOC108719307,gene1,id4
6,chr1L,Gnomon,exon,136318,136634,.,-,.,"transcript_id ""id4""; gene_id ""gene1""; gene_nam...","{'transcript_id': 'id4', 'gene_id': 'gene1', '...",LOC108719307,gene1,id4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1499202,MT,RefSeq,transcript,16249,17388,.,+,.,"transcript_id ""gene37282""; gene_id ""gene37282""...","{'transcript_id': 'gene37282', 'gene_id': 'gen...",CYTB,gene37282,gene37282
1499204,MT,RefSeq,transcript,17388,17457,.,+,.,"transcript_id ""rna62490""; gene_id ""rna62490""","{'transcript_id': 'rna62490', 'gene_id': 'rna6...",None,rna62490,rna62490
1499205,MT,RefSeq,exon,17388,17457,.,+,.,"transcript_id ""rna62490"";",{'transcript_id': 'rna62490'},None,None,rna62490
1499206,MT,RefSeq,transcript,17485,17553,.,-,.,"transcript_id ""rna62491""; gene_id ""rna62491""","{'transcript_id': 'rna62491', 'gene_id': 'rna6...",None,rna62491,rna62491


# 9. Generate gtf-idmm
This file maps the `gene_name` to `gene_id` and `transcript_id` fields generated by the conversion from GFF to GTF, which will be needed for downstream processing.

In [28]:
# Extract gene_name, gene_id, and transcript_id fields to generate an ID mapping matrix (idmm)
idmm_df = models_asgtf_df[['gene_name', 'gene_id', 'transcript_id']].drop_duplicates()
idmm_df.dropna(inplace = True)
display(idmm_df)

# generate a filename and file for the idmm
idmm_filename = '_'.join([species_prefix, conditions, 'gtf-idmm.tsv'])
idmm = IdmmFile(idmm_filename, species_SampleDict, kind = 'gtf_idmm', sources = [sample_Docket.annot])

# save to file and add to the Docket
idmm_df.to_csv(idmm.path, sep = '\t')
sample_Docket.add_keyfile(idmm, 'gtf_idmm')

,gene_name,gene_id,transcript_id
0,LOC108719298,gene0,id1
5,LOC108719307,gene1,id4
10,LOC108697125,gene2,rna0
17,LOC108696185,gene3,id10
22,LOC108696193,gene4,id13
...,...,...,...
1499186,ND4L,gene37278,gene37278
1499188,ND4,gene37279,gene37279
1499196,ND5,gene37280,gene37280
1499198,ND6,gene37281,gene37281


# 10. Generate updated gtf
Generated an updated GTF file using transcript_id as the key. For some datasets, transcripts do not consistently get gene names and gene IDs added, which causes Transdecoder to throw errors. This resolves that problem.

In [29]:
models_asgtf_updated_df = models_asgtf_df.merge(idmm_df, on = 'transcript_id')
models_asgtf_updated_df.apply(lambda x: x['field_dictionary'].update({'gene_name': x['gene_name_y']}), axis = 1)
models_asgtf_updated_df.apply(lambda x: x['field_dictionary'].update({'gene_id': x['gene_id_y']}), axis = 1)
models_asgtf_updated_df['new_fields'] = models_asgtf_updated_df['field_dictionary'].apply(convert_dict_to_fields_gtf)
models_asgtf_updated_df = models_asgtf_updated_df[[0, 1, 2, 3, 4, 5, 6, 7, 'new_fields']]

models_asgtf_updated_filename = models_asgtf.filename.replace('.gtf', '_updated.gtf')
models_asgtf_updated = GenomeGtfFile(models_asgtf_updated_filename, species_SampleDict, GenomeFastaFile = sample_Docket.genome_fasta)

models_asgtf_updated_df.to_csv(models_asgtf_updated.path, header = None, index = None, sep = '\t')

# 11. Generate cDNA and peptide files
Using the updated gtf file and genome file, generate cDNA sequence.

Then, using the cDNA sequence, generate peptide sequences using transdecoder.

Expect this step to take some time, probably ~20-30min.

In [30]:
cdna = sample_Docket.genome_fasta.get_transdecoder_cdna_gtf(models_asgtf_updated, TRANSDECODER_LOC)
sample_Docket.add_keyfile(cdna, 'cdna')

transdecoder_files = sample_Docket.cdna.to_pep_files(TDLONGORF_LOC, TDPREDICT_LOC)
sample_Docket.add_keyfiles(transdecoder_files)

-parsing cufflinks output: ../../output/Xlae_adultbrain/XENLA_9.2_GCF_updated.gtf
-parsing genome fasta: ../../output/Xlae_adultbrain/XENLA_9.2_genome.fa
-done parsing genome.
// processing Scaffold100
// processing Scaffold1000
// processing Scaffold100330
// processing Scaffold10049
// processing Scaffold10052
// processing Scaffold100627
// processing Scaffold101
// processing Scaffold101757
// processing Scaffold101947
// processing Scaffold101966
// processing Scaffold102
// processing Scaffold10219
// processing Scaffold1022
// processing Scaffold10281
// processing Scaffold102870
// processing Scaffold103
// processing Scaffold1032
// processing Scaffold103277
// processing Scaffold103338
// processing Scaffold103460
// processing Scaffold10353
// processing Scaffold10361
// processing Scaffold1037
// processing Scaffold104
// processing Scaffold10422
// processing Scaffold1045
// processing Scaffold105
// processing Scaffold1050
// processing Scaffold10527
// processing Scaffol

CMD: touch ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir/.__checkpoints_longorfs/TD.longorfs.ok




#################################
### Done preparing long ORFs.  ###
##################################

	Use file: ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//longest_orfs.pep  for Pfam and/or BlastP searches to enable homology-based coding region identification.

	Then, run TransDecoder.Predict for your final coding region predictions.


* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/get_top_longest_fasta_entries.pl ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//longest_orfs.cds 5000 5000 > ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//longest_orfs.cds.top_longest_5000
* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/exclude_similar_proteins.pl ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//longest_orfs.cds.top_longest_5000 > ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.tra

null device 
          1 


* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/compute_AUC.pl ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//start_refinement.feature.scores.roc


...

* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/make_seqLogo.Rscript ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//start_refinement.+.pwm || :
Error in library(seqLogo) : there is no package called ‘seqLogo’
Execution halted
* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/make_seqLogo.Rscript ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//start_refinement.-.pwm || :
Error in library(seqLogo) : there is no package called ‘seqLogo’
Execution halted
* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/deplete_feature_noise.pl  --features_plus ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//start_refinement.+.features  --pwm_minus ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//start_refinement.-.pwm  --out_prefix ../../output/Xlae_adultbrain/

null device 
          1 


* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/compute_AUC.pl ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//start_refinement.enhanced.feature.scores.roc


null device 
          1 


* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/PWM/make_seqLogo.Rscript ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//start_refinement.enhanced.+.pwm || :
Error in library(seqLogo) : there is no package called ‘seqLogo’
Execution halted
* Running CMD: /home/ec2-user/miniconda3/pkgs/transdecoder-5.5.0-pl526_1/opt/transdecoder/util/start_codon_refinement.pl --transcripts ../../output/Xlae_adultbrain/XENLA_9.2_genome_cDNA.fa --gff3_file ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//longest_orfs.cds.best_candidates.gff3 --workdir ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir/ > ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_dir//longest_orfs.cds.best_candidates.gff3.revised_starts.gff3
Refining start codon selections.
-number of revised start positions: 3129
* Running CMD: cp ../../output/Xlae_adultbrain/Xenopus_laevis_adultbrain.transdecoder_

# 12. Push files to AWS S3

Iteratively moves through the file_set and file_dict variables and populates files into the right place in AWS.

# 13. Pickle the `sample_Docket` variable for use by the next script

In [31]:
dill_filename = output_folder + '_'.join([species_prefix, conditions, 'sample_Docket.pkl'])

with open(dill_filename, 'wb') as file:
    dill.dump(sample_Docket, file)